In [2]:
import pandas as pd

Data Ingestion

In [15]:
raw_data = pd.read_csv('data/Transactions.csv')
raw_data.shape

(23053, 10)

In [16]:
raw_data.head()

,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type
0,80712190438,270351,28-02-2014,1,1,-5,-772,405.300,-4265.300,e-Shop
1,29258453508,270384,27-02-2014,5,3,-5,-1497,785.925,-8270.925,e-Shop
2,51750724947,273420,24-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop
3,93274880719,271509,24-02-2014,11,6,-3,-1363,429.345,-4518.345,e-Shop
4,51750724947,273420,23-02-2014,6,5,-2,-791,166.110,-1748.110,TeleShop


Data exploration

In [35]:
raw_data['total_amt'].describe()

count    23053.000000
mean      2107.308002
std       2507.561264
min      -8270.925000
25%        762.450000
50%       1754.740000
75%       3569.150000
max       8287.500000
Name: total_amt, dtype: float64

In [36]:
raw_data['tran_date'].describe()

count          23053
unique          1129
top       13-07-2011
freq              35
Name: tran_date, dtype: object

Necesary data transformations 

In [38]:
raw_data['tran_date'] = pd.to_datetime(raw_data['tran_date'])

In [39]:
raw_data['tran_date'].describe()

C:\Users\usuario\AppData\Local\Temp/ipykernel_20800/2136033722.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  raw_data['tran_date'].describe()


count                   23053
unique                   1129
top       2011-07-13 00:00:00
freq                       35
first     2011-01-02 00:00:00
last      2014-12-02 00:00:00
Name: tran_date, dtype: object

Function definition to obtain maximum date in order to calculate Recency

In [77]:
def pivotDate(df):
    d = raw_data.copy()
    d = d['tran_date'].max()
    d = pd.to_datetime(d)
    return d

In [78]:
pDate = pivotDate(raw_data)
pDate

Timestamp('2014-12-02 00:00:00')

Data aggregation to convert raw data to RFM data

In [53]:
rfm_data = raw_data.groupby('cust_id').agg({'tran_date': 'max', 'total_amt': 'sum', 'cust_id': 'size'})
rfm_data

,tran_date,total_amt,cust_id
cust_id,,,
266783,2013-09-02,3113.890,5
266784,2012-09-09,5694.065,3
266785,2013-02-13,21613.800,8
266788,2013-12-02,6092.970,4
266794,2014-12-02,27981.915,12
...,...,...,...
275257,2013-03-09,12574.900,5
275261,2013-06-27,442.000,5
275262,2012-02-29,5078.580,2


In [58]:
rfm_data.columns = ['Recency', 'Monetary', 'Frequency']
rfm_data

,Recency,Monetary,Frequency
cust_id,,,
266783,2013-09-02,3113.890,5
266784,2012-09-09,5694.065,3
266785,2013-02-13,21613.800,8
266788,2013-12-02,6092.970,4
266794,2014-12-02,27981.915,12
...,...,...,...
275257,2013-03-09,12574.900,5
275261,2013-06-27,442.000,5
275262,2012-02-29,5078.580,2


In [65]:
rfm_df = rfm_data.reset_index()
rfm_df = rfm_df[['cust_id', 'Recency', 'Frequency', 'Monetary']]
rfm_df

,cust_id,Recency,Frequency,Monetary
0,266783,2013-09-02,5,3113.890
1,266784,2012-09-09,3,5694.065
2,266785,2013-02-13,8,21613.800
3,266788,2013-12-02,4,6092.970
4,266794,2014-12-02,12,27981.915
...,...,...,...,...
5501,275257,2013-03-09,5,12574.900
5502,275261,2013-06-27,5,442.000
5503,275262,2012-02-29,2,5078.580
5504,275264,2011-08-10,2,3815.565


Now we need to transform date into months (calculating the difference between the max date and the pivot date)

In [69]:
def monthDiff(start_date, end_date):
    num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
    return num_months

In [79]:
rfm_df['Recency']  = rfm_df['Recency'].apply(lambda x : monthDiff(x, pDate))
rfm_df

,cust_id,Recency,Frequency,Monetary
0,266783,15,5,3113.890
1,266784,27,3,5694.065
2,266785,22,8,21613.800
3,266788,12,4,6092.970
4,266794,0,12,27981.915
...,...,...,...,...
5501,275257,21,5,12574.900
5502,275261,18,5,442.000
5503,275262,34,2,5078.580
5504,275264,40,2,3815.565


In [82]:
rfm_df.to_csv('data/RFM.csv', index=False)